In [1]:
using Distributions, CSV, DataFrames, ProgressMeter
include("../../utils_slv.jl")

MethodOfLines (generic function with 1 method)

In [2]:
T = 1.0; r=0.01; S0 = 100.0; v0 = 0.4; α = 0.65; β = 0.7; ρ = 0.65; K = 100.0; η=0.9; θ₀=0.02;
slvm = HestonSABRSLV(S0::Float64, v0::Float64, ρ::Float64, r::Float64, β::Float64, η::Float64, α::Float64, θ₀::Float64);
Π = [1.0 ρ; ρ 1.0]; cholΠ=cholesky(Π).L

drift(x) = [slvm.ω(abs(x[1]),abs(x[2])), slvm.μ(abs(x[2]))]
diffusion(x) = Diagonal(
        [slvm.m(abs(x[2]))*slvm.Γ(abs(x[1])), slvm.σ(abs(x[2]))]
        )*cholΠ
R = 0.07;
driver(t, x, y, z) = (
    -r*max(y-z'*inv(diffusion(x))*x, 0.0)
    -R*min(y-z'*inv(diffusion(x))*x, 0.0)
    -z'*inv(diffusion(x))*drift(x)
)
terminal(x) = max(K-x[1],0)
bsde = BSDE(T, [S0, v0], drift, diffusion, driver, terminal);

In [3]:
Nₜs = [10, 20, 50, 100, 200]
header = vcat(["scheme", "measurement_type"], string.(Nₜs))
df = DataFrame([[],[],[],[],[],[],[]], header)
schemes = [
    [LawsonEuler(krylov=true, m=100), true],
    [NorsettEuler(krylov=true, m=100), true],
    [ETDRK2(krylov=true, m=100), true],
    [ETDRK3(krylov=true, m=100), true],
    [ETDRK4(krylov=true, m=100), true],
    [HochOst4(krylov=true, m=100), true]
    # [DP5(), false],
    # [RadauIIA5(), false]
]

6-element Vector{Vector{Any}}:
 [LawsonEuler{0, true, Val{:forward}, true, nothing}(true, 100, 0), true]
 [NorsettEuler{0, true, Val{:forward}, true, nothing}(true, 100, 0), true]
 [ETDRK2{0, true, Val{:forward}, true, nothing}(true, 100, 0), true]
 [ETDRK3{0, true, Val{:forward}, true, nothing}(true, 100, 0), true]
 [ETDRK4{0, true, Val{:forward}, true, nothing}(true, 100, 0), true]
 [HochOst4{0, true, Val{:forward}, true, nothing}(true, 100, 0), true]

In [4]:
### designing grids
domain = [[1.0, 2*bsde.X0[1]-1.0], [bsde.X0[2]/15, 2*bsde.X0[2]-bsde.X0[2]/15]];
# domain = [[0.0, 2*bsde.X0[1]], [0.0, 2*bsde.X0[2]]];
Nₗ = [100, 15]; Δₗ = (bsde.X0-[dom[1] for dom in domain])./Nₗ;
Nᵣ = [100, 15]; Δᵣ = ([dom[2] for dom in domain]-bsde.X0)./Nᵣ;

grids = Array{AbstractGrid,1}(undef, 2)
g₁ = 1.0; g₂ = 1.0

grids[1] = TavellaRandallGrid(g₁, g₂, domain[1][1], bsde.X0[1], domain[1][2], Nₗ[1], Nᵣ[1])
grids[2] = Grid1D(
    vcat(domain[2][1]:Δₗ[2]:bsde.X0[2],(bsde.X0[2]+Δᵣ[2]):Δᵣ[2]:domain[2][2]),
    31,
    15,
    15
)

Grid1D([0.02666666666666667, 0.05155555555555556, 0.07644444444444445, 0.10133333333333334, 0.12622222222222224, 0.1511111111111111, 0.17600000000000002, 0.2008888888888889, 0.2257777777777778, 0.2506666666666667  …  0.5493333333333335, 0.5742222222222223, 0.5991111111111111, 0.6240000000000001, 0.648888888888889, 0.6737777777777778, 0.6986666666666668, 0.7235555555555557, 0.7484444444444446, 0.7733333333333334], 31, 15, 15)

In [5]:
d=2
@showprogress for attr in schemes
    abs100 = zeros(Float64, length(Nₜs)); 
    runtimes = zeros(Float64, length(Nₜs))
    scheme = attr[1]
    EXPINT = attr[2]
    @showprogress for (ind, Nₜ) in enumerate(Nₜs)
        print(string("Nₜ=", Nₜ,":\n")); flush(stdout)
        exc_start = time()
        res = MethodOfLines(bsde, grids, Nₜ, scheme, EXPINT)
        exc_stop = time()
        sol = res[1]; s_grid = res[2]; 
        
        index = grids[1].Nₗ*grids[2].N + grids[2].Nₗ + 1
        abs100[ind] = sol[index,end]
        runtimes[ind] = exc_stop - exc_start
        print(string("Y0(100,0.4)=", abs100[ind],":\n")); flush(stdout)
        print(string("Runtime[s]=", runtimes[ind],":\n")); flush(stdout)
    end
    schemename = split(split(string(scheme), '{')[1], '(')[1]
    row_100 = vcat([schemename, "Y0(100,0.4)"], string.(abs100)); push!(df,row_100);
    row_run = vcat([schemename, "Runtime[s]"], string.(runtimes)); push!(df,row_run);
end
df |> CSV.write(string("result_1029.csv"))

Nₜ=10:
Y0(100,0.4)=4.798466278661286:
Runtime[s]=22.7935528755188:
Nₜ=20:
Y0(100,0.4)=5.560529933987746:
Runtime[s]=29.70417618751526:


Progress:  40%|████████████████▍                        |  ETA: 0:01:19

Nₜ=50:
Y0(100,0.4)=5.601496927099452:
Runtime[s]=66.14965796470642:


Progress:  60%|████████████████████████▋                |  ETA: 0:01:19

Nₜ=100:
Y0(100,0.4)=5.621065483140017:
Runtime[s]=129.12831282615662:


...

Nₜ=100:
Y0(100,0.4)=5.639276277750687:
Runtime[s]=551.4694700241089:


Progress:  80%|████████████████████████████████▊        |  ETA: 0:04:13

Nₜ=200:
Y0(100,0.4)=5.6393771702059405:
Runtime[s]=1117.3502159118652:


Progress: 100%|█████████████████████████████████████████| Time: 0:35:29
Progress: 100%|█████████████████████████████████████████| Time: 2:04:11


"result_1029.csv"